In [46]:
! git clone https://github.com/cyizhuo/CUB-200-2011-dataset.git

fatal: destination path 'CUB-200-2011-dataset' already exists and is not an empty directory.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

In [ ]:
# Inception-ResNet V2 in PyTorch
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [ ]:
class InceptionResNetV2(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionResNetV2, self).__init__()

        self.features = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, stride=2),
            BasicConv2d(32, 32, kernel_size=3),
            BasicConv2d(32, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2),
            BasicConv2d(64, 80, kernel_size=1),
            BasicConv2d(80, 192, kernel_size=3),
            nn.MaxPool2d(kernel_size=3, stride=2),
            InceptionResNetBlock(192, scale=0.17),
            InceptionResNetBlock(192, scale=0.17),
            InceptionResNetBlock(192, scale=0.17),
            InceptionResNetBlock(192, scale=0.17),
            InceptionResNetBlock(192, scale=0.17),
        )

        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(192, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.global_avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [ ]:
class InceptionResNetBlock(nn.Module):
    def __init__(self, in_channels, scale=1.0):
        super(InceptionResNetBlock, self).__init__()
        self.branch0 = BasicConv2d(in_channels, 32, kernel_size=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(in_channels, 32, kernel_size=1),
            BasicConv2d(32, 32, kernel_size=3, padding=1)
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, 32, kernel_size=1),
            BasicConv2d(32, 48, kernel_size=3, padding=1),
            BasicConv2d(48, 64, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1),
            BasicConv2d(in_channels, 64, kernel_size=1)
        )

        self.scale = scale

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat([x0, x1, x2, x3], 1)
        out = out * self.scale + x
        return F.relu(out)


In [ ]:
# Instantiate the model
model = InceptionResNetV2()

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
])

## Trial 2 : InceptionV3

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import models
import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# https://pytorch.org/docs/stable/torchvision/models.html
# https://github.com/pytorch/vision/tree/master/torchvision/models
# 미리 사용할 모델의 Input 파악 필수!

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = dsets.ImageFolder('/content/CUB-200-2011-dataset/train', train_transform)
test_data = dsets.ImageFolder('/content/CUB-200-2011-dataset/test', test_transform)

In [ ]:
batch_size = 5

train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(test_data,
                         batch_size=batch_size,
                         shuffle=True)

In [ ]:
def imshow(img, title):
    img = torchvision.utils.make_grid(img, normalize=True)
    npimg = img.numpy()
    fig = plt.figure(figsize = (5, 15))
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
dataiter = iter(train_loader)
images,labels = next(dataiter)

imshow(images, [train_data.classes[i] for i in labels])

In [ ]:
model = models.inception_v3(pretrained=True)

In [ ]:
# Count the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters in the model: {total_params}")

In [ ]:
model

In [ ]:
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

In [ ]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 200),
    nn.Linear(200, 200)
)

In [54]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
model = model.to(device)

In [55]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [57]:
num_epochs = 30
for epoch in range(num_epochs):

    total_batch = len(train_data)//batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):

        X = batch_images.to(device)
        Y = batch_labels.to(device)

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/30], lter [5/1198] Loss: 10.6910
Epoch [1/30], lter [10/1198] Loss: 7.8522
Epoch [1/30], lter [15/1198] Loss: 5.3572
Epoch [1/30], lter [20/1198] Loss: 5.6290
Epoch [1/30], lter [25/1198] Loss: 5.5459
Epoch [1/30], lter [30/1198] Loss: 5.4429
Epoch [1/30], lter [35/1198] Loss: 5.9395
Epoch [1/30], lter [40/1198] Loss: 6.3657
Epoch [1/30], lter [45/1198] Loss: 5.2014
Epoch [1/30], lter [50/1198] Loss: 5.5290
Epoch [1/30], lter [55/1198] Loss: 5.6559
Epoch [1/30], lter [60/1198] Loss: 5.9758
Epoch [1/30], lter [65/1198] Loss: 5.7673
Epoch [1/30], lter [70/1198] Loss: 5.9051
Epoch [1/30], lter [75/1198] Loss: 5.4446
Epoch [1/30], lter [80/1198] Loss: 5.7331
Epoch [1/30], lter [85/1198] Loss: 6.2610
Epoch [1/30], lter [90/1198] Loss: 5.4560
Epoch [1/30], lter [95/1198] Loss: 5.4248
Epoch [1/30], lter [100/1198] Loss: 5.4642
Epoch [1/30], lter [105/1198] Loss: 5.8285
Epoch [1/30], lter [110/1198] Loss: 5.3596
Epoch [1/30], lter [115/1198] Loss: 5.0237
Epoch [1/30], lter [120/1198] 

KeyboardInterrupt: 

In [60]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:

    images = images.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.to(device)).sum()

print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 31.532620 %


In [ ]:
classes = ["Black_footed_Albatross", "Laysan_Albatross"]

In [ ]:
images, labels = iter(test_loader).next()

outputs = model(images.to(device))

_, predicted = torch.max(outputs.data, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(5)))

title = (' '.join('%5s' % classes[labels[j]] for j in range(5)))
imshow(torchvision.utils.make_grid(images, normalize=True), title)